<a href="https://colab.research.google.com/github/noushadkhan01/Projects/blob/master/Hackathons/Analytics%20Vidhya/Build%20a%20Recommender%20Engine%20Powered%20by%20IBM%20Cloud/Recommender%20Engine%20Powered%20by%20IBM%20Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [64]:
train = pd.read_csv('https://bit.ly/32Okxx9')
train.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,27270,27270,7,01/12/10 8:26,2.55,PX,85123AY
1,27270,27270,7,01/12/10 8:26,3.39,PX,71053R
2,27270,27270,9,01/12/10 8:26,2.75,PX,84406BH
3,27270,27270,7,01/12/10 8:26,3.39,PX,84029GV
4,27270,27270,7,01/12/10 8:26,3.39,PX,84029EX


In [4]:
train.shape

(330575, 7)

In [79]:
test = pd.read_csv('https://bit.ly/30SBLrj')
test.head()

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [6]:
train.shape, test.shape

((330575, 7), (103097, 7))

In [7]:
train.isnull().sum().sort_values(ascending = False)[:2]

StockCode    0
Country      0
dtype: int64

In [8]:
test.isnull().sum().sort_values(ascending = False)[:2]

UnitPrice    0
StockCode    0
dtype: int64

**As we can see train and test dataset have differend order for columns so we need to reorder this**

In [9]:
train.columns

Index(['CustomerID', 'InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'Country', 'StockCode'],
      dtype='object')

In [10]:
test.columns

Index(['Country', 'CustomerID', 'InvoiceDate', 'InvoiceNo', 'Quantity',
       'StockCode', 'UnitPrice'],
      dtype='object')

In [11]:
test = test[train.columns]
test.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,127269,127269,7,01/12/10 8:28,1.85,PX,22633V
1,227268,227268,38,01/12/10 8:34,1.69,PX,84879M
2,227268,227268,7,01/12/10 8:34,2.10,PX,22748P
3,227268,227268,9,01/12/10 8:34,3.75,PX,22749K
4,227268,227268,2,01/12/10 8:34,9.95,PX,22622G


In [12]:
test.columns

Index(['CustomerID', 'InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'Country', 'StockCode'],
      dtype='object')

**no of unique countries**

In [13]:
train.Country.nunique()

38

**no. of unique products**

In [14]:
train.StockCode.nunique(), test.StockCode.nunique()

(3810, 3522)

In [15]:
test.InvoiceDate.dtype

dtype('O')

In [0]:
def check_invoice_count_per_customer(data):
  id_g = data.groupby('CustomerID')
  yes = 0
  for i, j in id_g:
    uinvoice = j.InvoiceNo.nunique()
    if uinvoice > 1:
      print(i)
      print(uinvoice, '\n\n')
      yes += 1
  if not yes:
    print('none of the customers have bought more than 1 time')

In [17]:
check_invoice_count_per_customer(train)

none of the customers have bought more than 1 time


In [18]:
check_invoice_count_per_customer(test)

none of the customers have bought more than 1 time


In [19]:
train.columns

Index(['CustomerID', 'InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'Country', 'StockCode'],
      dtype='object')

**Check Quantity column**

In [20]:
train.Quantity.unique()[:50]

array([  7,   9,   2,   3,  28,  14,  57,  21,  24,  43,  38,   4,  76,
        12, 144, 115,  27,   5,   1,  48, 120, 230, 518,  19,  10,  30,
        36,  33,   8, 240,  86, 720, 576, 172,  -6,  96,  60,  -1,  -2,
        13,  39,  15,  84, 153, 302,  72, 259, 460,  16, 129])

In [21]:
train.Quantity.min(), train.Quantity.max()

(-9600, 97194)

In [22]:
test.Quantity.min(), test.Quantity.max()

(-80995, 5160)

**As we can see Quantity column have negative value so we need to replace it with postive sign**

In [0]:
train['Quantity'] = train.Quantity.apply(lambda x: -x if x < 0 else x)
test['Quantity'] = test.Quantity.apply(lambda x: -x if x < 0 else x)

In [24]:
train.Quantity.min(), train.Quantity.max()

(1, 97194)

In [25]:
test.Quantity.min(), test.Quantity.max()

(1, 80995)

##drop less important columns

In [26]:
train.columns

Index(['CustomerID', 'InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'Country', 'StockCode'],
      dtype='object')

In [27]:
test.columns

Index(['CustomerID', 'InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'Country', 'StockCode'],
      dtype='object')

In [0]:
train.drop(['InvoiceNo', 'InvoiceDate', 'UnitPrice'], inplace = True, axis = 1)
test.drop(['InvoiceNo', 'InvoiceDate', 'UnitPrice'], inplace = True, axis = 1)

In [29]:
train.head(1)

,CustomerID,Quantity,Country,StockCode
0,27270,7,PX,85123AY


In [30]:
test.head(1)

,CustomerID,Quantity,Country,StockCode
0,127269,7,PX,22633V


#unique products in store

In [31]:
products = pd.concat([train.StockCode, test.StockCode], axis = 0)
products.shape

(433672,)

In [32]:
products = products.unique()
products.shape

(3860,)

In [33]:
train.StockCode.nunique()

3810

In [34]:
test.StockCode.nunique()

3522

#Combine Dataset for pivot table

In [38]:
combined = pd.concat([train, test], axis = 0)
combined.shape

(433672, 4)

In [39]:
combined = combined.pivot_table(index = 'CustomerID', columns = ['StockCode', 'Country'], values = 'Quantity')
combined.head()

StockCode  10002F                  ... 90214UO 90214VD 90214WH 90214YL 90214ZJ
Country        GJ  IG  PX  RA  SK  ...      PX      PX      PX      PX      PX
CustomerID                         ...                                        
0             NaN NaN NaN NaN NaN  ...     NaN     NaN     NaN     NaN     NaN
900           NaN NaN NaN NaN NaN  ...     NaN     NaN     NaN     NaN     NaN
1800          NaN NaN NaN NaN NaN  ...     NaN     NaN     NaN     NaN     NaN
1890          NaN NaN NaN NaN NaN  ...     NaN     NaN     NaN     NaN     NaN
2700          NaN NaN NaN NaN NaN  ...     NaN     NaN     NaN     NaN     NaN

[5 rows x 17864 columns]

In [40]:
l = set(train.CustomerID.unique()) - set(test.CustomerID.unique())
len(l)

972

In [41]:
train.CustomerID.nunique(), test.CustomerID.nunique()

(972, 628)

In [77]:
train = combined.loc[train.CustomerID.unique()]
train.shape

(972, 17864)

In [94]:
test = combined.loc[test.CustomerID.unique()]
test.shape

(628, 17864)

#Cosine Similarity

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
test.shape

In [0]:
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

In [50]:
similarity = cosine_similarity(test, train)
similarity

array([[0.01380899, 0.00995712, 0.00748101, ..., 0.        , 0.00610045,
        0.        ],
       [0.09205441, 0.05575506, 0.00364617, ..., 0.        , 0.02648811,
        0.        ],
       [0.10524991, 0.03469774, 0.01942362, ..., 0.        , 0.01692498,
        0.        ],
       ...,
       [0.        , 0.00361499, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0024693 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00906813, 0.01006702, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [72]:
max_similarity = similarity.argsort()
max_similarity

array([[971, 842, 845, ..., 904,  61, 565],
       [971, 528, 965, ..., 334, 735, 578],
       [971, 209, 953, ..., 734, 626, 174],
       ...,
       [  0, 613, 615, ..., 325, 338, 565],
       [  0, 623, 625, ..., 814, 911, 762],
       [485, 610, 601, ..., 372, 905, 822]])

In [73]:
max_similarity = max_similarity[:, -1]
max_similarity[:10]

array([565, 578, 174,  12, 256, 913, 578, 582, 359, 459])

In [66]:
train_for_submission = pd.read_csv('https://bit.ly/32Okxx9')
train_for_submission.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,27270,27270,7,01/12/10 8:26,2.55,PX,85123AY
1,27270,27270,7,01/12/10 8:26,3.39,PX,71053R
2,27270,27270,9,01/12/10 8:26,2.75,PX,84406BH
3,27270,27270,7,01/12/10 8:26,3.39,PX,84029GV
4,27270,27270,7,01/12/10 8:26,3.39,PX,84029EX


In [92]:
test_for_submission = pd.read_csv('https://bit.ly/30SBLrj')
test_for_submission.head()

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [0]:
train_for_submission['Quantity'] = train_for_submission.Quantity.apply(lambda x: -x if x < 0 else x)
test_for_submission['Quantity'] = test_for_submission.Quantity.apply(lambda x: -x if x < 0 else x)

In [0]:
max_customer_id = []
n = 0
for i in max_similarity:
  max_customer_id.append((test.index[n], train.index[i]))
  n += 1

In [124]:
max_customer_id[:10]

[(127269, 526365),
 (227268, 256437),
 (327267, 178218),
 (162738, 147249),
 (252747, 354537),
 (347247, 23769),
 (442755, 256437),
 (342756, 453546),
 (57240, 348246),
 (357237, 435456)]

In [102]:
n = 0
for i, j in max_customer_id.items():
  print(i, ':--', j)
  if n > 10:
    break
  n += 1

127269 :-- 526365
227268 :-- 256437
327267 :-- 178218
162738 :-- 147249
252747 :-- 354537
347247 :-- 23769
442755 :-- 256437
342756 :-- 453546
57240 :-- 348246
357237 :-- 435456
531765 :-- 471915
131769 :-- 532665


In [105]:
train_pivot = train_for_submission.pivot_table(index = 'CustomerID', columns = 'StockCode', values = 'Quantity')
train_pivot.head()

StockCode,10002F,10080Q,10120M,10123CU,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,15060bU,16008I,16010H,16011L,16012X,16014F,16015Z,16016H,16020CF,16033Z,16043M,16045M,...,90204H,90205AR,90205CD,90206AQ,90206CD,90208D,90209BE,90209CQ,90210AL,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90212CC,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214UO,90214VD,90214WH,90214YL,90214ZJ
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
test_pivot = test_for_submission.pivot_table(index = 'CustomerID', columns = 'StockCode', values = 'Quantity')
test_pivot.head()

StockCode,10002F,10080Q,10120M,10124AW,10124GF,10125J,10133M,10135P,11001M,15030G,15034D,15036Z,15039A,15044AR,15044BI,15044CR,15044DX,15056BLH,15056ND,15056PV,15056blG,15056nO,15056pC,15058AY,15058BR,15058CH,15060BB,16008I,16011L,16012X,16014F,16015Z,16016H,16020CF,16043M,16045M,16046R,16048K,16049D,16052Q,...,90202AZ,90202BJ,90202CT,90202DB,90204H,90205AR,90205CD,90206CD,90208D,90209AX,90209BE,90209CQ,90210BN,90210CN,90210DP,90211AP,90211BD,90212BG,90214AQ,90214BW,90214CK,90214DZ,90214EF,90214FX,90214GF,90214HZ,90214IZ,90214JA,90214KK,90214LO,90214MZ,90214NM,90214OZ,90214PB,90214RZ,90214SF,90214TT,90214VD,90214YL,90214ZJ
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3600,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4500,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
train_group = train_for_submission.groupby('CustomerID')
test_group = test_for_submission.groupby('CustomerID')

In [111]:
df = pd.DataFrame(np.arange(10))
df

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In [141]:
test_id_recommendation = []
for x in max_customer_id:
  i, j = x
  train_id_products = train_group.get_group(j)[['Quantity', 'StockCode']]
  test_id_products = test_group.get_group(i)[['Quantity', 'StockCode']]
  test_stock = set(test_id_products.StockCode)
  train_stock = set(train_id_products.StockCode)
  diff_stock = train_stock - test_stock
  diff = list(diff_stock)
  if len(diff) == 0:
    print(i, j)
  diff_train_products = train_id_products.set_index('StockCode').loc[diff].sort_values(by = 'Quantity', ascending = False).index
  test_id_recommendation.append((i, diff_train_products[0]))
  

531765 471915


IndexError: ignored

In [144]:
test_group.get_group(531765)['StockCode'].shape, train_group.get_group(471915)['StockCode'].shape

((256,), (3,))

In [146]:
test_group.get_group(531765)[['Quantity', 'StockCode']]

,Quantity,StockCode
64,14,22969Z
17195,7,22470E
17196,9,22429T
17197,3,22427Q
17198,7,22804V
17199,7,85123AY
19702,1,22312A
19703,1,84659AF
19704,7,22419N
19705,7,21640F


In [145]:
train_group.get_group(471915)[['Quantity', 'StockCode']]

,Quantity,StockCode
166483,38,85123AY
166484,7,22720A
166485,60,47566Y


In [0]:
train_id_products = train_group.get_group(j)[['Quantity', 'StockCode']]
test_id_products = test_group.get_group(i)[['Quantity', 'StockCode']]
test_stock = set(test_id_products.StockCode)
train_stock = set(train_id_products.StockCode)
diff_stock = train_stock - test_stock
diff = list(diff_stock)
diff[:10]
diff_train_products = train_id_products.set_index('StockCode').loc[diff].sort_values(by = 'Quantity', ascending = False).index
diff_train_products
test_id_recommendation.append((i, diff_train_products[0]))

In [138]:
test_id_recommendation

[(127269, '48185J')]